setup

In [ ]:
import pandas as pd
from datetime import datetime
start_time = datetime.now()

set up sorter

In [ ]:
def sort_key(word):

    pāli_alphabet = [
        "√", "a", "ā", "i", "ī", "u", "ū", "e", "o", "k", "kh", "g", "gh", "ṅ",
        "c", "ch", "j", "jh", "ñ", "ṭ", "ṭh", "ḍ", "ḍḥ", "ṇ", "t", "th", "d",
        "dh", "n", "p", "ph", "b", "bh", "m", "y", "r", "l", "s", "v", "h",
        "ḷ", "ṃ", " ", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0"
    ]

    #comp iteration condition
    dl = [i for i in pāli_alphabet if len(i) > 1]

    for i in dl:
        word = word.replace(i, '/{}'.format(i))

    wordVe = []

    k = -3

    for j in range(len(word)):
        if word[j] == '/':
            k = j
            wordVe.append(word[j + 1:j + 3])
        if j > k + 2:
            wordVe.append(word[j])

    word = wordVe

    pāli_alphabet_string = '-'.join(pāli_alphabet)
    return [pāli_alphabet_string.find('-' + x + '-') for x in wordVe]

setup roots df

In [ ]:
roots_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/roots.csv", sep="\t", dtype=str)
roots_df.fillna("", inplace=True)

# roots_df.replace("\.0$", "", inplace=True, regex=True)
roots_df = roots_df[roots_df["Count"] != "0"] # remove roots with no examples
roots_df = roots_df[roots_df["Fin"] != ""] # remove extra iines
roots_df.sort_values(by = ["Root"], inplace=True, ignore_index=True, key=lambda x: x.map(sort_key)) #sort
roots_df["Dhātupātha"] = roots_df["Dhātupātha"].str.replace("-", "")
roots_df["Kaccāyana Dhātu Mañjūsā"] = roots_df["Kaccāyana Dhātu Mañjūsā"].str.replace("-", "")

roots_df_count = roots_df.shape[0]

setup dpd_df

In [ ]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True)

dpd_df.loc[dpd_df["Meaning IN CONTEXT"] == "", "Meaning IN CONTEXT"] = dpd_df["Buddhadatta"] + "*"

extract families from dpd-full.csv

In [ ]:
test1 = dpd_df["Family"] != ""
test2 = dpd_df["Metadata"] == ""
filter = test1 & test2
root_families_df = dpd_df.loc[filter, ["Pāli Root", "Grp", "Root Meaning", "Family"]]

root_families_df = root_families_df.drop_duplicates(subset=["Pāli Root", "Grp", "Root Meaning", "Family"])
root_families_df.sort_values(["Pāli Root", "Grp", "Root Meaning", "Family"], ascending = (True, True, True, True), inplace=True)
root_families_df.sort_values(by = ["Family"], inplace=True, ignore_index=True, key=lambda x: x.map(sort_key)) #sort
root_families_df = root_families_df.reset_index(drop=True)

# row count
root_families_df_count = root_families_df.shape[0]

generate html for each root subfamily

In [ ]:
print("~"*40)
print("generating html for each root subfamily")
print("~"*40)

for row in range(root_families_df_count):

	root = root_families_df.loc[row, "Pāli Root"]
	root_group = root_families_df.loc[row, "Grp"]
	root_meaning = root_families_df.loc[row, "Root Meaning"]
	subfamily = root_families_df.loc[row, "Family"]

	if row % 500 == 0 or row /  root_families_df_count == 1:
		print(f"{row}/{root_families_df_count}\t{subfamily} {root_group} {root_meaning}")

	# dpd_df.columns
	test1 = dpd_df["Pāli Root"] == root
	test2 = dpd_df["Grp"] == root_group
	test3 = dpd_df["Root Meaning"] == root_meaning
	test4 = dpd_df["Family"] == subfamily
	filter = test1 & test2 & test3 & test4
	subfamily_df = dpd_df.loc[filter, ["Pāli1", "POS", "Meaning IN CONTEXT"]]

	subfamily_df_length = subfamily_df.shape[0]

	html_string = ""

	html_string += """<tbody>"""
		
	for row_sf in range(subfamily_df_length):
		sf_pali = subfamily_df.iloc[row_sf, 0]
		sf_pos = subfamily_df.iloc[row_sf, 1]
		sf_english = subfamily_df.iloc[row_sf, 2]

		html_string += f"<tr><th>{sf_pali}</th>"
		html_string += f"<td>{sf_pos}</td>"
		html_string += f"<td>{sf_english}</td></tr>"
	
	html_string += f"""</tbody>"""

	with open(f"output/subfamily html/{root} {root_group} {root_meaning} {subfamily}.html", "w") as output_file:
		output_file.write(html_string)

extract bases

In [ ]:
print("~"*40)
print("extracting bases")
print("~"*40)

bases_df = dpd_df

bases_dict = {}

for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_group = roots_df.iloc[row, 5]
	root_meaning = roots_df.iloc[row, 8]

	test1 = bases_df["Pāli Root"] == root
	test2 = bases_df["Grp"] == root_group
	test3 = bases_df["Root Meaning"] == root_meaning
	test4 = bases_df["Base"].str.contains(fr">")

	filter = test1 & test2 & test3 & test4
	bases_filtered = bases_df.loc[filter, ["Base"]]

	bases_filtered.drop_duplicates("Base", inplace=True, keep="first")
	bases_filtered = bases_filtered.dropna()
	bases_filtered = bases_filtered.sort_values(by="Base", key=lambda x: x.str.len())
	bases_filtered = bases_filtered["Base"].str.replace("^.+ > ", "")

	bases_filtered_size = bases_filtered.shape[0]

	if row % 100 == 0:
		print(f"{row}/{roots_df_count}\t{root} {root_group} {root_meaning}")

	with open(f"output/bases/{root} {root_group} {root_meaning}.csv", "w") as output_file:
		bases_filtered_size = bases_filtered.shape[0]

		if bases_filtered_size == 0:
			output_file.write(f"-")
		if bases_filtered_size > 0:
			bases_filtered.to_csv(output_file, header=False, index=False, sep="\t")

extract families to csv

In [ ]:
root_families_df = dpd_df

for row in range(roots_df_count):
	count = roots_df.iloc[row, 1]
	root = roots_df.iloc[row, 2]
	root_group = roots_df.iloc[row, 5]
	root_meaning = roots_df.iloc[row, 8]

	test1 = root_families_df["Pāli Root"] == root
	test2 = root_families_df["Root Meaning"] == root_meaning
	test3 = root_families_df["Grp"] == root_group
	filter = test1 & test2 & test3
	root_families_filtered = root_families_df.loc[filter, ["Family"]]
	root_families_filtered.drop_duplicates("Family", inplace=True, keep='first')

	root_families_filtered.sort_values(by = ["Family"], inplace=True, ignore_index=True, key=lambda x: x.map(sort_key)) #sort

	with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/families/{root} {root_group} {root_meaning}.csv", "w") as output_file:
		root_families_filtered.to_csv(output_file, header=False, index=False, sep="\t")

write root information

In [ ]:
print("~"*40)
print("writing root info")


import re

for row in range(roots_df_count):
	root = roots_df.iloc[row, 2]
	root_in_comps = roots_df.iloc[row, 3]
	root_has_verb = roots_df.iloc[row, 4]
	root_group = roots_df.iloc[row, 5]
	root_sign = roots_df.iloc[row, 6]
	root_meaning = roots_df.iloc[row, 8]

	base_file = open(f"output/bases/{root} {root_group} {root_meaning}.csv")
	base = base_file.read()
	base_file.close()
	base = re.sub("\n", ", ", base)
	base = re.sub(", $", "", base)

	sk_root = roots_df.iloc[row, 9]
	sk_root_meaning = roots_df.iloc[row, 10]
	sk_root_class = roots_df.iloc[row, 11]
	example = roots_df.iloc[row, 12]
	dhp_no = roots_df.iloc[row, 13]
	dhp_root = roots_df.iloc[row, 14]
	dhp_meaning = roots_df.iloc[row, 15]
	dhp_english = roots_df.iloc[row, 16]
	dhm_no = roots_df.iloc[row, 17]
	dhm_root = roots_df.iloc[row, 18]
	dhm_meaning = roots_df.iloc[row, 19]
	dhm_english = roots_df.iloc[row, 20]
	sdn_root = roots_df.iloc[row, 21]
	sdn_meaning = roots_df.iloc[row, 22]
	sdn_english = roots_df.iloc[row, 23]
	pdp_root = roots_df.iloc[row, 24]
	pdp_meaning = roots_df.iloc[row, 25]
	pdp_english = roots_df.iloc[row, 26]
	notes = roots_df.iloc[row, 27]

	if str(root_group) == "1":
		root_group_pali = "bhūvādigaṇa"
	if str(root_group) == "2":
		root_group_pali = "rudhādigaṇa"
	if str(root_group) == "3":
		root_group_pali = "divādigaṇa"
	if str(root_group) == "4":
		root_group_pali = "svādigaṇa"
	if str(root_group) == "5":
		root_group_pali = "kiyādigaṇa"
	if str(root_group) == "6":
		root_group_pali = "gahādigaṇa"
	if str(root_group) == "7":
		root_group_pali = "tanādigaṇa"
	if str(root_group) == "8":
		root_group_pali = "curādigaṇa"


	html_string = ""
	html_string += f"""<tbody>"""
	html_string += f"""<tr><th>pāli root:</th><td>{root}<sup>{root_has_verb}</sup>{root_group} {root_group_pali} +{root_sign} ({root_meaning})</td></tr>"""
	html_string += f"""<tr><th>base(s):</th><td>{base}</td></tr>"""

# Root in comps
	if root_in_comps != "":
		html_string += f"""<tr><th>in compounds:</th><td>{root_in_comps}</td></tr>"""
	else:
		pass

# Dhātupātha
	if dhp_root != "-":
		html_string += f"""<tr><th>dhātupātha:</th><td>{dhp_root} <i>{dhp_meaning}</i> ({dhp_english}) #{dhp_no}</td></tr>"""
	else:
		html_string += f"""<tr><th>dhātupātha:</th><td>-</td></tr>"""

	# Dhātumañjūsa
	if dhm_root != "-":
		html_string += f"""<tr><th>dhātumañjūsa:</th><td>{dhm_root} <i>{dhm_meaning}</i> ({dhm_english}) #{dhm_no}</td></tr>"""
	else:
		html_string += f"""<tr><th>dhātumañjūsa:</th><td>-</td></tr>"""

	# Saddanīti
	if sdn_root != "-":
		html_string += f"""<tr><th>saddanīti:</th><td>{sdn_root} <i>{sdn_meaning}</i> ({sdn_english})</td></tr>"""
	else:
		html_string += f"""<tr><th>saddanīti:</th><td>-</td></tr>"""

	# Sanskrit
	html_string += f"""<tr><th>sanskrit root:</th><td style = 'color:gray'>{sk_root} {sk_root_class} ({sk_root_meaning})</td></tr>"""

	# Pāṇinīya Dhātupāṭha
	if pdp_root != "-":
		html_string += f"""<tr><th>pāṇinīya dhātupāṭha:</th><td style = 'color:gray'>{pdp_root} <i>{pdp_meaning}</i> ({pdp_english})</td></tr>"""
	else:
		html_string += f"""<tr><th>pāṇinīya dhātupāṭha:</th><td>-</td></tr>"""

	html_string += f"""</tbody>"""

	with open(f"output/root info/{root} {root_group} {root_meaning}.html", "w") as output_file:
		output_file.write(html_string)

timer

In [ ]:
print("~"*40)
end_time = datetime.now()
print('{} subtotal'.format(end_time - start_time))